## Day 21

https://adventofcode.com/2021/day/21

In [254]:
def detDie():
    roll = 1
    while True:
        yield roll
        roll = roll % 100 + 1

def matchDeterministic(board):
    score = [0,0]
    die = detDie()
    nrolls = 0
    p = 0
    while max(score)<1000:
        # roll dice 3 times
        dieTrows = sum([ next(die) for _ in range(3) ])
        nrolls += 3
        # move on board, increase score accordingly
        board[p] = ( board[p] + dieTrows -1 ) % 10 + 1
        score[p] += board[p]
        # change player
        p = (p+1)%2
    return nrolls*min(score)

In [255]:
# Player 1 starting position: 4
# Player 2 starting position: 8
board = [4,8]
matchDeterministic(board)

739785

In [256]:
# Player 1 starting position: 10
# Player 2 starting position: 2
board = [10,2]
matchDeterministic(board)

916083

In [257]:
from itertools import product
from functools import lru_cache

@lru_cache(maxsize=None)
def matchDirac(board_,score_,p):
    win = [0,0]
    
    # all possibilities for the 3 trows of current player
    for trows in product([1,2,3],repeat=3):
        
        dieTrows = sum(trows)
        
        # local mutable copies of board and score for current trow sequence
        board = list(board_)
        score = list(score_)
        
        board[p] = ( board[p] + dieTrows -1 ) % 10 + 1
        score[p] += board[p]

        if score[p]>=21:
            win[p] += 1
        else:
            pnew = (p+1)%2 # don't reuse p since the trow possibility for current player might not be exhausted!
            # pass board and score to next universe/player as unmutable
            theWin = matchDirac(tuple(board),tuple(score),pnew)
            for i in range(2):
                win[i] += theWin[i]

    return win

In [258]:
# player 1 wins in 444356092776315
# player 2 wins in 341960390180808

board = [4,8]
score = [0,0]
matchDirac(tuple(board),tuple(score),0) # use tuple() since list are not hashable and I'm using caching

[444356092776315, 341960390180808]

In [259]:
board = [10,2]
score = [0,0]
win = matchDirac(tuple(board),tuple(score),0)
print(win, max(win))

[49982165861983, 36086577212020] 49982165861983


In [263]:
# since the sum of all possible 3 trows is not unique, 
# I can speed up even further by grouping the universes by trow sum

from collections import Counter

sumTrows = [ sum(trows) for trows in product([1, 2, 3], repeat=3) ]
sumTrowCount = Counter(sumTrows)

@lru_cache(maxsize=None)
def matchDiracBetter(board_,score_,p):
    win = [0,0]
    
    for dieTrows,counts in sumTrowCount.items():
        
        # local copies of board and score for trow sequence
        board = list(board_)
        score = list(score_)
        
        board[p] = ( board[p] + dieTrows -1 ) % 10 + 1
        score[p] += board[p]

        if score[p]>=21:
            win[p] += counts
        else:
            pnew = (p+1)%2
            theWin = matchDiracBetter(tuple(board),tuple(score),pnew)
            for i in range(2):
                win[i] += theWin[i]*counts

    return win

In [264]:
board = [4,8]
score = [0,0]
matchDiracBetter(tuple(board),tuple(score),0)

[444356092776315, 341960390180808]

In [265]:
board = [10,2]
score = [0,0]
win = matchDiracBetter(tuple(board),tuple(score),0)
print(win, max(win))

[49982165861983, 36086577212020] 49982165861983
